# **QAI Robotic Command + RL Motion Planner Demo**

Python simulation for Colab that demonstrates the basic behavior of a QAI-like robotic system:

    1) Takes input from a user (client command)

    2) Analyzes and classifies the command

    3) Simulates action (with mock environment response)

    4) Provides feedback

    5) Adapts logic using basic reinforcement logic

In [4]:
# QAI DEMO: Multi-Command → Parser → QPU Offload → Feedback + RL Motion Planner
import random
import time
import numpy as np

print("========== QAI COMMAND PARSER + QPU OFFLOAD ==========")

# Simulated command library
command_library = {
    "inspect": "Inspect object using vision system",
    "assemble": "Assemble part using robotic gripper",
    "weld": "Weld components with controlled heat",
    "navigate": "Start RL-based motion planner to reach target"
}

# QAI parser: natural command → logical intent
def qai_parser(input_cmd):
    keywords = {
        "inspect": ["inspect", "check", "scan"],
        "assemble": ["assemble", "put", "build"],
        "weld": ["weld", "join", "fuse"],
        "navigate": ["navigate", "go", "move", "target"]
    }
    for k, v in keywords.items():
        if any(word in input_cmd.lower() for word in v):
            return k
    return "unknown"

# Fake QPU offloader
def qpu_offload(task):
    print(f"\n[QPU] Offloading task: {task}")
    time.sleep(1)
    outcome = random.choices(["success", "unstable", "fail"], weights=[0.7, 0.2, 0.1])[0]
    print(f"[QPU] Task executed with result: {outcome}")
    return outcome

# Feedback loop
confidence = {cmd: 1 for cmd in command_library}
def feedback_update(cmd, result):
    if result == "success":
        confidence[cmd] += 1
    else:
        confidence[cmd] = max(0, confidence[cmd] - 1)
    print(f"[QAI] Updated confidence for '{cmd}': {confidence[cmd]}")

# RL motion planner block
def run_rl_motion_planner():
    print("\n========== RL MOTION PLANNER INITIATED ==========")
    grid = np.array([
        [0, 0, 0, 0],
        [0, 1, 1, 0],
        [0, 0, 0, 9]
    ])
    start_pos = (0, 0)
    goal_pos = (2, 3)
    actions = ['up', 'down', 'left', 'right']
    Q = np.zeros((3, 4, 4))
    alpha = 0.5
    gamma = 0.9
    epsilon = 0.2

    def move(pos, action):
        x, y = pos
        if action == 'up': x -= 1
        elif action == 'down': x += 1
        elif action == 'left': y -= 1
        elif action == 'right': y += 1
        if 0 <= x < 3 and 0 <= y < 4 and grid[x][y] != 1:
            return (x, y)
        return pos

    def reward(pos):
        return 10 if pos == goal_pos else -1

    pos = start_pos
    print(f"🚶 Starting at {pos}")
    for step in range(20):
        x, y = pos
        a = random.randint(0, 3) if random.random() < epsilon else np.argmax(Q[x, y])
        action = actions[a]
        next_pos = move(pos, action)
        r = reward(next_pos)
        nx, ny = next_pos
        Q[x, y, a] = Q[x, y, a] + alpha * (r + gamma * np.max(Q[nx, ny]) - Q[x, y, a])
        print(f"[STEP {step}] At {pos}, action: {action} → {next_pos} | Reward: {r}")
        pos = next_pos
        if pos == goal_pos:
            print("🎯 Goal Reached!")
            break

# === MULTI-COMMAND EXECUTION ===
commands = input("🤖 Client Commands (comma separated, e.g., 'inspect panel, weld joint, navigate to goal'): ")
for user_input in commands.split(','):
    user_input = user_input.strip()
    cmd_key = qai_parser(user_input)

    if cmd_key in command_library:
        print(f"\n[QAI] Parsed intent: {cmd_key} → {command_library[cmd_key]}")
        if cmd_key != "navigate":
            result = qpu_offload(cmd_key)
            feedback_update(cmd_key, result)
        else:
            run_rl_motion_planner()
    else:
        print(f"[QAI] Command '{user_input}' unrecognized.")


========== QAI COMMAND PARSER + QPU OFFLOAD ==========
🤖 Client Commands (comma separated, e.g., 'inspect panel, weld joint, navigate to goal'): 'inspect panel, weld joint, navigate to goal'

[QAI] Parsed intent: inspect → Inspect object using vision system

[QPU] Offloading task: inspect
[QPU] Task executed with result: success
[QAI] Updated confidence for 'inspect': 2

[QAI] Parsed intent: weld → Weld components with controlled heat

[QPU] Offloading task: weld
[QPU] Task executed with result: success
[QAI] Updated confidence for 'weld': 2

[QAI] Parsed intent: navigate → Start RL-based motion planner to reach target

========== RL MOTION PLANNER INITIATED ==========
🚶 Starting at (0, 0)
[STEP 0] At (0, 0), action: up → (0, 0) | Reward: -1
[STEP 1] At (0, 0), action: down → (1, 0) | Reward: -1
[STEP 2] At (1, 0), action: right → (1, 0) | Reward: -1
[STEP 3] At (1, 0), action: up → (0, 0) | Reward: -1
[STEP 4] At (0, 0), action: left → (0, 0) | Reward: -1
[STEP 5] At (0, 0), action: r

**Concept and execution by Bhadale IT, code generated by ChatGPT**